<a href="https://colab.research.google.com/github/RobertoCuellar2019/tesis_1/blob/master/v1_reconstruccion_no_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**VERSIÓN 1.0 CÓDIGO TESIS DE PREGRADO**

RECONSTRUCCIÓN DE OBJETOS 3D UTILIZANDO HOLOGRAMAS FRACCIONARES Y REDES NEURONALES PARA EL RECONOCIMIENTO DE PATRONES DE DIFRACCIÓN

ROBERTO ANTONIO CUELLAR LOZANO
UNIVERSIDAD DE PAMPLONA 
PAMPLONA- NORTE DE SANTANDER 


In [0]:
import numpy as np  # libreria de calculo numerico y arreglos
import matplotlib.pyplot as pl # libreria para visualización